In [1]:
import pandas as pd
import time
from datetime import datetime
from gpt4all import GPT4All
import os

# === CONFIG ===
BASE_DIR = r"C:\Users\ROYAUME MEDIAS\OneDrive\Desktop\Similarity"
MODEL_PATH = os.path.join(BASE_DIR, "Meta-Llama-3-8B-Instruct.Q4_0.gguf")
DATA_PATH = os.path.join(BASE_DIR, "job_title_des_cleaned.csv")
OUTPUT_PATH = os.path.join(BASE_DIR, "JobDescription_WithDetails.csv")

# === LOAD MODEL ===
print("🔄 Loading GPT4All model (local CPU)...")
model = GPT4All(
    model_name=MODEL_PATH,
    model_type="llama",
    allow_download=False
)
print("✅ Model loaded successfully!\n")

# === LOAD DATA ===
df_job = pd.read_csv(DATA_PATH)
print(f"📄 Loaded {len(df_job)} job entries.\n")

# === FUNCTION TO GENERATE JOB DETAILS ===
def generate_job_details(job_title, job_description):
    today = datetime.now().strftime("%B %d, %Y")
    prompt = f"""
    You are a professional HR specialist. Write a detailed, structured, and realistic job description for the following role:

    **Job Title:** {job_title}
    **Date:** {today}

    Use the following info as a base:
    {job_description}

    Include the following sections:
    1. Company Overview
    2. Job Mission
    3. Key Responsibilities (4–8 points)
    4. Required Skills & Qualifications
    5. Preferred Skills
    6. Tools & Technologies
    7. Education
    8. Work Environment
    9. Benefits
    10. Application Instructions

    Make it professional, clear, and formatted like a LinkedIn or Indeed job post.
    """
    with model.chat_session():
        generated_desc = model.generate(prompt, max_tokens=900)
    return generated_desc.strip()

# === APPLY GPT4ALL TO FIRST 9 JOBS ONLY ===
job_details_list = []

for idx, row in df_job.head(9).iterrows():  # Limité aux 9 premiers jobs
    job_title = row.get('Job Title', 'Unknown Position')
    job_description = row.get('Job Description', '')
    print(f"💼 Generating details for job {idx}: {job_title}")
    start_time = time.time()
    details = generate_job_details(job_title, job_description)
    end_time = time.time()
    print(f"✅ Done in {end_time - start_time:.2f} sec\n")
    job_details_list.append(details)

# === ADD NEW COLUMN (ONLY FOR FIRST 9 JOBS) ===
df_job.loc[:8, 'JobDetails'] = job_details_list  # lignes 0 à 8

# === SAVE NEW CSV ===
df_job.to_csv(OUTPUT_PATH, index=False)
print(f"\n💾 Job details for first 9 jobs saved to: {OUTPUT_PATH}")


🔄 Loading GPT4All model (local CPU)...
✅ Model loaded successfully!

📄 Loaded 2277 job entries.

💼 Generating details for job 0: flutter,developer
✅ Done in 333.06 sec

💼 Generating details for job 1: django,developer
✅ Done in 297.67 sec

💼 Generating details for job 2: machine,learning
✅ Done in 475.71 sec

💼 Generating details for job 3: io,developer
✅ Done in 447.06 sec

💼 Generating details for job 4: full,stack,developer
✅ Done in 1188.29 sec

💼 Generating details for job 5: java,developer
✅ Done in 497.74 sec

💼 Generating details for job 6: full,stack,developer
✅ Done in 653.88 sec

💼 Generating details for job 7: javascript,developer
✅ Done in 494.47 sec

💼 Generating details for job 8: devops,engineer
✅ Done in 1089.31 sec


💾 Job details for first 9 jobs saved to: C:\Users\ROYAUME MEDIAS\OneDrive\Desktop\Similarity\JobDescription_WithDetails.csv


In [5]:
import sqlite3
import pandas as pd

# === CONFIG ===
CSV_PATH = "JobDescription_WithDetails.csv"
DB_PATH = "jobs.db"

# === CHARGER LE CSV ===
df = pd.read_csv(CSV_PATH)

# === FONCTION DE DÉTECTION DU DOMAINE ===
def get_domain(title):
    t = title.lower()
    if any(x in t for x in ["ai", "machine learning", "ml", "deep learning", "nlp", "computer vision"]):
        return "AI"
    if any(x in t for x in ["data", "big data", "data engineer", "data analyst", "data scientist"]):
        return "Data"
    if any(x in t for x in ["backend", "java", "python", "golang", "node", "php", "ruby"]):
        return "Backend"
    if any(x in t for x in ["frontend", "react", "angular", "vue", "javascript", "typescript", "ui"]):
        return "Frontend"
    if any(x in t for x in ["fullstack", "full stack"]):
        return "Fullstack"
    if any(x in t for x in ["flutter", "ios", "android", "mobile"]):
        return "Mobile"
    if any(x in t for x in ["devops", "docker", "kubernetes", "k8s", "ci/cd"]):
        return "DevOps"
    if any(x in t for x in ["cloud", "aws", "azure", "gcp"]):
        return "Cloud"
    if any(x in t for x in ["security", "cyber", "pentest", "soc"]):
        return "Security"
    if any(x in t for x in ["network", "infrastructure", "system administrator", "sysadmin"]):
        return "Infrastructure"
    if any(x in t for x in ["qa", "test", "testing", "automation"]):
        return "QA / Testing"
    if any(x in t for x in ["scrum", "agile", "project manager"]):
        return "Management"
    if any(x in t for x in ["sap", "erp", "odoo"]):
        return "ERP"
    if any(x in t for x in ["embedded", "iot", "microcontroller"]):
        return "Embedded / IoT"
    return "Other"

# === CONNEXION À SQLITE ===
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# === CRÉER LA TABLE SI ELLE N'EXISTE PAS ===
cursor.execute("""
CREATE TABLE IF NOT EXISTS jobs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    description TEXT,
    domain TEXT
)
""")

# === INSÉRER LES 50 PREMIÈRES LIGNES ===
for idx, row in df.head(50).iterrows():
    title = row.get("Job Title", "").replace(",", " ").title()
    description = row.get("JobDetails", "")
    domain = get_domain(title)

    cursor.execute("""
    INSERT INTO jobs (title, description, domain)
    VALUES (?, ?, ?)
    """, (title, description, domain))

conn.commit()
conn.close()

print(f"✅ {len(df.head(50))} jobs insérés dans SQLite ({DB_PATH})")


✅ 50 jobs insérés dans SQLite (jobs.db)


In [ ]:
# venv\Scripts\activate


In [3]:
%pip install pymongo



  Using cached dnspython-2.8.0-py3-none-any.whl.metadata (5.7 kB)
   ---------------------------------------- 0.0/859.2 kB ? eta -:--:--
   ------------ --------------------------- 262.1/859.2 kB ? eta -:--:--
   ------------------------ --------------- 524.3/859.2 kB 2.8 MB/s eta 0:00:01
   ------------------------------------ --- 786.4/859.2 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 859.2/859.2 kB 1.2 MB/s  0:00:00
Using cached dnspython-2.8.0-py3-none-any.whl (331 kB)

   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   ------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install "gpt4all[cuda]"


   ---------------------------------------- 0.0/427.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/427.2 MB 5.6 MB/s eta 0:01:17
   ---------------------------------------- 1.6/427.2 MB 4.9 MB/s eta 0:01:27
   ---------------------------------------- 2.1/427.2 MB 4.7 MB/s eta 0:01:31
   ---------------------------------------- 3.1/427.2 MB 4.1 MB/s eta 0:01:44
   ---------------------------------------- 3.7/427.2 MB 3.8 MB/s eta 0:01:51
   ---------------------------------------- 4.2/427.2 MB 4.0 MB/s eta 0:01:46
   ---------------------------------------- 5.2/427.2 MB 4.0 MB/s eta 0:01:46
    --------------------------------------- 6.3/427.2 MB 3.8 MB/s eta 0:01:52
    --------------------------------------- 7.1/427.2 MB 3.9 MB/s eta 0:01:47
    --------------------------------------- 7.9/427.2 MB 3.8 MB/s eta 0:01:51
    --------------------------------------- 8.4/427.2 MB 3.7 MB/s eta 0:01:54
    --------------------------------------- 8.9/427.2 MB 3.6 MB/s eta 0


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import shutil
import os

# Chemin actuel du modèle
MODEL_PATH = r"C:\Users\ROYAUME MEDIAS\OneDrive\Desktop\Similarity\Meta-Llama-3-8B-Instruct.Q4_0.gguf"

# Nouveau chemin / nouveau nom du modèle
NEW_MODEL_PATH = r"C:\Users\ROYAUME MEDIAS\OneDrive\Desktop\Similarity\Meta-Llama-3-8B-Instruct-Copy.gguf"

# Vérifier si le fichier existe
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Le modèle original n'a pas été trouvé : {MODEL_PATH}")

# Copier le modèle
shutil.copy2(MODEL_PATH, NEW_MODEL_PATH)
print(f"✅ Modèle copié avec succès sous : {NEW_MODEL_PATH}")


✅ Modèle copié avec succès sous : C:\Users\ROYAUME MEDIAS\OneDrive\Desktop\Similarity\Meta-Llama-3-8B-Instruct-Copy.gguf
